10. 번역기를 만들어보자

<<목차>>

1) 정제, 정규화, 전처리 (영어, 프랑스어 모두!)  

2) 디코더의 문장에 시작 토큰과 종료 토큰을 넣어주세요.

3) 케라스의 토크나이저로 텍스트를 숫자로 바꿔보세요.

4) 임베딩 층(Embedding layer) 사용하기

5) 모델 구현하기

6) 모델 평가하기

7) 루브릭

8) 회고

https://www.manythings.org/anki/

사용할 데이터는  프랑스어와 영어의 병렬 코퍼스인 fra-eng.zip

In [36]:
import pandas as pd
import tensorflow as tf
import numpy as np
import os
import re

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, Embedding, Masking, LSTM, Dense, Dropout
from tensorflow.keras.models import Model


1) 정제, 정규화, 전처리

In [37]:
file_path = '/home/june/data/translator_seq2seq/fra.txt'
lines = pd.read_csv(file_path, names=['eng', 'fra', 'cc'], sep='\t')
print("전체 샘플의 수 : ", len(lines))
lines.sample(5)

전체 샘플의 수 :  197463


,eng,fra,cc
173979,My painting is starting to look pretty cool.,Mon tableau commence à avoir de la gueule.,CC-BY 2.0 (France) Attribution: tatoeba.org #1...
97255,Not all of them are present.,Ils ne sont pas tous présents.,CC-BY 2.0 (France) Attribution: tatoeba.org #3...
90416,There is a man at the door.,Il y a un homme à la porte.,CC-BY 2.0 (France) Attribution: tatoeba.org #3...
88665,I'm living with my parents.,Je vis avec mes parents.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
169890,Tom recommends that you not do that again.,Tom vous conseille de ne plus recommencer.,CC-BY 2.0 (France) Attribution: tatoeba.org #6...


In [38]:
lines = lines[['eng', 'fra']][:50000] # 5만개 샘플 사용
lines.sample(5)

,eng,fra
35656,I can't do it alone.,Je ne peux le faire seul.
11510,No one laughed.,Personne n'a ri.
33662,Why don't you quit?,Pourquoi n'arrêtez-vous pas ?
49835,I know how this works.,Je sais comment cela fonctionne.
16913,What's the idea?,Quelle est l'idée ?


In [39]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()
  
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    sentence = re.sub(r"[^a-zA-Z?.!]+", " ", sentence)
    
    sentence = sentence.strip()
    sentence = sentence.split(" ")
    
    return sentence

In [40]:
def preprocess_sentence_decoder(sentence):
    sentence = sentence.lower().strip()
  
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    sentence = re.sub(r"[^a-zA-Z?.!]+", " ", sentence)
    
    sentence = sentence.strip()
    sentence = '<start> ' + sentence + ' <end>'
    sentence = sentence.split(" ")
    
    return sentence

In [41]:
lines.eng = lines.eng.apply(lambda x : preprocess_sentence(x))

lines.eng.sample(5)

26411        [this, is, his, fault, .]
41073    [do, you, know, his, name, ?]
34827    [don, t, do, that, to, me, .]
27208         [we, were, impressed, .]
21794         [what, did, you, say, ?]
Name: eng, dtype: object

In [42]:
lines.fra = lines.fra.apply(lambda x : preprocess_sentence_decoder(x))

lines.fra.sample(5)

44789       [<start>, elle, a, parl, impoliment, ., <end>]
18214    [<start>, il, alla, faire, des, emplettes, ., ...
21471    [<start>, nous, pouvons, encore, gagner, ., <e...
13434           [<start>, vous, fiez, vous, moi, ?, <end>]
29388    [<start>, comment, pouvais, tu, savoir, ?, <end>]
Name: fra, dtype: object

3) 케라스의 토크나이저로 텍스트를 숫자로 바꿔보세요.

In [43]:
eng_tokenizer = Tokenizer()
eng_tokenizer.fit_on_texts(lines.eng)
input_text = eng_tokenizer.texts_to_sequences(lines.eng)
input_text[:5]

[[33, 1], [33, 1], [33, 1], [33, 1], [908, 1]]

In [44]:
fra_tokenizer = Tokenizer()
fra_tokenizer.fit_on_texts(lines.fra)
target_text = fra_tokenizer.texts_to_sequences(lines.fra)
target_text[:5]

[[1, 89, 15, 2],
 [1, 458, 3, 2],
 [1, 30, 649, 15, 2],
 [1, 935, 15, 2],
 [1, 936, 15, 2]]

In [45]:
eng_vocab_size = len(eng_tokenizer.word_index) + 1
fra_vocab_size = len(fra_tokenizer.word_index) + 1
print('영어 단어장의 크기 :', eng_vocab_size)
print('프랑스어 단어장의 크기 :', fra_vocab_size)

영어 단어장의 크기 : 5987
프랑스어 단어장의 크기 : 9313


In [46]:
max_eng_seq_len = max([len(line) for line in input_text])
max_fra_seq_len = max([len(line) for line in target_text])
print('영어 시퀀스의 최대 길이', max_eng_seq_len)
print('프랑스어 시퀀스의 최대 길이', max_fra_seq_len)

영어 시퀀스의 최대 길이 9
프랑스어 시퀀스의 최대 길이 19


In [47]:
print('전체 샘플의 수 :',len(lines))
print('영어 단어장의 크기 :', eng_vocab_size)
print('프랑스어 단어장의 크기 :', fra_vocab_size)   
print('영어 시퀀스의 최대 길이', max_eng_seq_len)
print('프랑스어 시퀀스의 최대 길이', max_fra_seq_len)

전체 샘플의 수 : 50000
영어 단어장의 크기 : 5987
프랑스어 단어장의 크기 : 9313
영어 시퀀스의 최대 길이 9
프랑스어 시퀀스의 최대 길이 19


In [48]:
sos_token = '<start>'
eos_token = '<end>'

encoder_input = input_text
# 종료 토큰 제거
decoder_input = [[ char for char in line if char != fra_tokenizer.word_index[eos_token] ] for line in target_text] 
# 시작 토큰 제거
decoder_target = [[ char for char in line if char != fra_tokenizer.word_index[sos_token] ] for line in target_text]

In [49]:
encoder_input = pad_sequences(encoder_input, maxlen = max_eng_seq_len, padding='post')
decoder_input = pad_sequences(decoder_input, maxlen = max_fra_seq_len, padding='post')
decoder_target = pad_sequences(decoder_target, maxlen = max_fra_seq_len, padding='post')
print('영어 데이터의 크기(shape) :',np.shape(encoder_input))
print('프랑스어 입력데이터의 크기(shape) :',np.shape(decoder_input))
print('프랑스어 출력데이터의 크기(shape) :',np.shape(decoder_target))

영어 데이터의 크기(shape) : (50000, 9)
프랑스어 입력데이터의 크기(shape) : (50000, 19)
프랑스어 출력데이터의 크기(shape) : (50000, 19)


In [50]:
indices = np.arange(encoder_input.shape[0])
np.random.shuffle(indices)

encoder_input = encoder_input[indices]
decoder_input = decoder_input[indices]
decoder_target = decoder_target[indices]

In [51]:
n_of_val = 3000

encoder_input_train = encoder_input[:-n_of_val]
decoder_input_train = decoder_input[:-n_of_val]
decoder_target_train = decoder_target[:-n_of_val]

encoder_input_test = encoder_input[-n_of_val:]
decoder_input_test = decoder_input[-n_of_val:]
decoder_target_test = decoder_target[-n_of_val:]

print(encoder_input_train.shape)
print(decoder_input_train.shape)
print(decoder_target_train.shape)
print(encoder_input_test.shape)
print(decoder_input_test.shape)
print(decoder_target_test.shape)

(47000, 9)
(47000, 19)
(47000, 19)
(3000, 9)
(3000, 19)
(3000, 19)


4) 임베딩 층(Embedding layer) 사용하기

In [52]:
embedding_size = 512
hidden_size = 512

encoder_inputs = Input(shape=(None, ), name='encoder_input')
enc_emb =  Embedding(eng_vocab_size, embedding_size,
                    input_length=max_eng_seq_len)(encoder_inputs)
enc_masking = Masking(mask_value=0.0)(enc_emb)
encoder_lstm = LSTM(hidden_size, dropout = 0.5, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_masking)
encoder_states = [state_h, state_c]

In [53]:
decoder_inputs = Input(shape=(None, ), name='decoder_input')
dec_emb =  Embedding(fra_vocab_size, embedding_size)(decoder_inputs)
dec_masking = Masking(mask_value=0.0)(dec_emb)
decoder_lstm = LSTM(hidden_size, dropout = 0.5, return_sequences = True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_masking, initial_state = encoder_states)

In [54]:
decoder_softmax_layer = Dense(fra_vocab_size, activation='softmax')
decoder_outputs = decoder_softmax_layer(decoder_outputs)

In [55]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 encoder_input (InputLayer)     [(None, None)]       0           []                               
                                                                                                  
 decoder_input (InputLayer)     [(None, None)]       0           []                               
                                                                                                  
 embedding_5 (Embedding)        (None, None, 512)    3065344     ['encoder_input[0][0]']          
                                                                                                  
 embedding_6 (Embedding)        (None, None, 512)    4768256     ['decoder_input[0][0]']          
                                                                                            

In [56]:
model.compile(optimizer="rmsprop", loss='sparse_categorical_crossentropy', metrics=['acc'])

In [57]:
model.fit(x=[encoder_input_train, decoder_input_train], 
          y=decoder_target_train, 
          validation_data = ([encoder_input_test, decoder_input_test], 
                             decoder_target_test),
          batch_size=32, 
          epochs=50)

Epoch 1/50


2022-10-25 22:32:42.214425: W tensorflow/core/common_runtime/forward_type_inference.cc:332] Type inference failed. This indicates an invalid graph that escaped type checking. Error message: INVALID_ARGUMENT: expected compatible input types, but input 1:
type_id: TFT_OPTIONAL
args {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_TENSOR
    args {
      type_id: TFT_INT32
    }
  }
}
 is neither a subtype nor a supertype of the combined inputs preceding it:
type_id: TFT_OPTIONAL
args {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_TENSOR
    args {
      type_id: TFT_FLOAT
    }
  }
}

	while inferring type of node 'cond_40/output/_23'
2022-10-25 22:32:43.388914: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8600
2022-10-25 22:32:43.730857: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


1469/1469 [==============================] - 34s 19ms/step - loss: 1.2493 - acc: 0.8048 - val_loss: 1.0370 - val_acc: 0.8344
Epoch 2/50
1469/1469 [==============================] - 27s 18ms/step - loss: 0.9488 - acc: 0.8464 - val_loss: 0.8922 - val_acc: 0.8545
Epoch 3/50
1469/1469 [==============================] - 26s 18ms/step - loss: 0.8302 - acc: 0.8626 - val_loss: 0.8097 - val_acc: 0.8679
Epoch 4/50
1469/1469 [==============================] - 26s 18ms/step - loss: 0.7533 - acc: 0.8743 - val_loss: 0.7569 - val_acc: 0.8763
Epoch 5/50
1110/1469 [=====================>........] - ETA: 6s - loss: 0.6991 - acc: 0.8839

5) 모델 구현하기

In [ ]:
encoder_model = Model(inputs = encoder_inputs, outputs = encoder_states)
encoder_model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding (Embedding)       (None, None, 256)         1532672   
                                                                 
 masking (Masking)           (None, None, 256)         0         
                                                                 
 lstm (LSTM)                 [(None, 256),             525312    
                              (None, 256),                       
                              (None, 256)]                       
                                                                 
Total params: 2,057,984
Trainable params: 2,057,984
Non-trainable params: 0
_________________________________________________________________


In [ ]:
decoder_state_input_h = Input(shape=(embedding_size,))
decoder_state_input_c = Input(shape=(embedding_size,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2 = Embedding(fra_vocab_size, embedding_size)(decoder_inputs)
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state = decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]

decoder_outputs2 = decoder_softmax_layer(decoder_outputs2)

In [ ]:
decoder_model = Model(inputs=[decoder_inputs] + decoder_states_inputs, outputs=[decoder_outputs2] + decoder_states2)
decoder_model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 decoder_input (InputLayer)     [(None, None)]       0           []                               
                                                                                                  
 embedding_4 (Embedding)        (None, None, 512)    4768256     ['decoder_input[0][0]']          
                                                                                                  
 input_3 (InputLayer)           [(None, 512)]        0           []                               
                                                                                                  
 input_4 (InputLayer)           [(None, 512)]        0           []                               
                                                                                            

In [ ]:
def decode_sequence(input_seq):
    # 입력으로부터 인코더의 상태를 얻음
    states_value = encoder_model.predict(input_seq)

    # <start>에 해당하는 원-핫 벡터 생성
    target_seq = np.zeros((1,1))
    target_seq[0, 0] = fra2idx['<start>']
    
    stop_condition = False
    decoded_sentence = ""

    # stop_condition이 True가 될 때까지 루프 반복
    while not stop_condition:
        # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # 예측 결과를 문자로 변환
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = idx2fra[sampled_token_index]

        # 현재 시점의 예측 문자를 예측 문장에 추가
        decoded_sentence += ' '+sampled_char

        # <eos>에 도달하거나 최대 길이를 넘으면 중단.
        if (sampled_char == '<end>' or
           len(decoded_sentence) > max_fra_seq_len):
            stop_condition = True

        # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index

        # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
        states_value = [h, c]

    return decoded_sentence

In [ ]:
# 원문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq2src(input_seq):
    temp=''
    for i in input_seq:
        if(i!=0):
            temp = temp + idx2eng[i]+' '
    return temp

In [ ]:
# 번역문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq2tar(input_seq):
    temp=''
    for i in input_seq:
        if((i!=0 and i!=fra2idx['<start>']) and i!=fra2idx['<end>']):
            temp = temp + idx2fra[i] + ' '
    return temp

6) 모델 평가하기

In [ ]:
for seq_index in [1,200,500,1001,2022]:
    input_seq = encoder_input_test[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print(35 * "-")
    print('입력 문장:', seq2src(encoder_input_test[seq_index]))
    print('정답 문장:', seq2tar(decoder_input_test[seq_index]))
    print('번역기가 번역한 문장:', decoded_sentence[:len(decoded_sentence)-1])

1/1 [==============================] - 0s 18ms/step
-----------------------------------
입력 문장: i quit my job . 
정답 문장: j ai d missionn de mon boulot . 
번역기가 번역한 문장:  j mon mon mon ma . c
1/1 [==============================] - 0s 16ms/step
-----------------------------------
입력 문장: he let me go . 
정답 문장: il m a laiss partir . 
번역기가 번역한 문장:  il m m m . . . . . 
1/1 [==============================] - 0s 20ms/step
-----------------------------------
입력 문장: i sat on the sofa . 
정답 문장: je me suis assis sur le canap . 
번역기가 번역한 문장:  je s e e me me on 
1/1 [==============================] - 0s 17ms/step
-----------------------------------
입력 문장: i dropped my earring . 
정답 문장: j ai fait tomber ma boucle d oreille . 
번역기가 번역한 문장:  j me me une une un
1/1 [==============================] - 0s 16ms/step
-----------------------------------
입력 문장: she did pretty well . 
정답 문장: elle s en est assez bien sortie . 
번역기가 번역한 문장:  elle voulez avez bie


1. 번역기 모델 학습에 필요한 텍스트 데이터 전처리가 잘 이루어졌다.

구두점, 대소문자, 띄어쓰기 등 번역기 모델에 요구되는 전처리가 정상적으로 진행되었다.


2. seq2seq 기반의 번역기 모델이 정상적으로 구동된다.

seq2seq 모델 훈련결과를 그래프로 출력해보고, validation loss그래프가 우하향하는 경향성을 보이며 학습이 진행됨이 확인되었다.


3. 테스트 결과 의미가 통하는 수준의 번역문이 생성되었다.

테스트용 디코더 모델이 정상적으로 만들어졌으며, input(영어)와 output(프랑스어) 모두 한글로 번역해서 결과를 출력해보았고, 둘의 내용이 유사함을 확인하였다.

8) 회고

- 이번 프로젝트에서 **어려웠던 점,**




- 프로젝트를 진행하면서 **알아낸 점** 혹은 **아직 모호한 점**




- 루브릭 평가 지표를 맞추기 위해 **시도한 것들**




- 만약에 루브릭 평가 관련 지표를 **달성 하지 못했을 때, 이유에 관한 추정**


- **자기 다짐**

